**Data cleaning** is the process of fixing or removing incorrect, corrupted, incorrectly formatted, duplicate, or incomplete data within a dataset. When combining multiple data sources, there are many opportunities for data to be duplicated or mislabeled. If data is incorrect, outcomes and algorithms are unreliable, even though they may look correct. There is no one absolute way to prescribe the exact steps in the data cleaning process because the processes will vary from dataset to dataset. But it is crucial to establish a template for your data cleaning process so you know you are doing it the right way every time. In this section, I will clean my text data and modified data. The steps for cleaning these two kind of data is different, so I will specify the cleaning steps with presenting the code. 

## Python on Text Data 

#### Dataset "twitterpython.csv" which is grabbed through python

People's perception of energy consumption is very important. It not only reflects the public's perception, but also indirectly reflects people's concern for environmental issues and their willingness to change the status quo. In the data gathering section, I have gather the text data by setting “energy consumption” “global warming” “clean energy” “conventional energy” “environmental protection” as the keywords for the twitter api to get the text data. Now I will do the data processing for the text dataset through python. 

 - Step 1: import the packages we may use during data processing

In [ ]:
import pandas as pd
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn import svm
import nltk
import warnings
warnings.filterwarnings('ignore')
from nltk.sentiment import SentimentIntensityAnalyzer

 - Step 2: Load the Dataset we grab by using python

In [ ]:
twitter = pd.read_csv("/Users/apple/Desktop/anly-501-project-HuitingSong-1/501-project-website/data/raw data/twitterpython.csv")
twitter.head()

 - Step 3: Find the dataset with NA missing value
 
 Through checking, there is no missing value in the text dataset.


In [ ]:
twitter.isnull().sum()

 - Step 4: Remove data with language other than English

 Since my Twitter API collect the text with different languages, I need to drop the text information that cannot be understand by me and you. I only keep the text with English.

In [ ]:
twitter = twitter[twitter.lang == "en"]
twitter

 - Step 5: Pre-processing the text info in the dataset
 
 Since the raw data contains lots of puntuations, urls, commas, numbers, highercase, and some other things which will influence our analysis and tokenization. Therefore, we need to remove the unnecessary things. Since this semester, I take Natural Language Processing course, and learn how to use **pipline** to process the text information. So, in this part, I will the method I learn from NLP. 
 
 Through pipline, the things which will influence our analysis will be cleaned and replaced. And then, we will need to transform the sentence from text data to tokens which is a list of words through tokenizer.


In [ ]:
import re
import spacy
from spacy.language import Language

pipeline = spacy.load('en_core_web_sm')

# http://emailregex.com/
email_re = r"""(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*|"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])"""

# replace = [ (pattern-to-replace, replacement),  ...]
replace = [
    (r"<a[^>]*>(.*?)</a>", r"\1"),  # Matches most URLs
    (email_re, "email"),            # Matches emails
    (r"(?<=\d),(?=\d)", ""),        # Remove commas in numbers
    (r"\d+", "number"),              # Map digits to special token <numbr>
    (r"[\t\n\r\*\.\@\,\-\/]", " "), # Punctuation and other junk
    (r"\s+", " ")                   # Stips extra whitespace
]

twitter_sentences = []
for i, d in enumerate(twitter['text']):
    for repl in replace:
        d = re.sub(repl[0], repl[1], d)
    twitter_sentences.append(d)

@Language.component("lab04Preprocessor")
def ng20_preprocess(doc):
    tokens = [token for token in doc 
              if not any((token.is_stop, token.is_punct))]
    tokens = [token.lemma_.lower().strip() for token in tokens]
    tokens = [token for token in tokens if token]
    return " ".join(tokens)


pipeline.add_pipe("lab04Preprocessor")

 - Step 6: Pass data through spacy pipeline

In [ ]:
docs = []
for sent in twitter_sentences:
    docs.append(pipeline(sent))

result = pd.DataFrame(docs)
result.head()

 - Step 7: Backup the processed data

In [ ]:
result.to_csv("/Users/apple/Desktop/anly-501-project-HuitingSong-1/501-project-website/data/cleaned data/piplineresult.csv")

 - Step 8: Using the CountVectorizer and count the words freqency
 
 Countvectorizer can convert a collection of text documents to a matrix of token counts.

In [ ]:
vectorizer = CountVectorizer()
bow = vectorizer.fit(docs)
features = bow.vocabulary_.keys()
counts = bow.vocabulary_.values()
bow=pd.DataFrame({'words':features,'counts':counts})
bow = bow.sort_values(by=['counts'],ascending=False)
bow

 - Step 9: Backup the processed data 

In [ ]:
bow.to_csv("/Users/apple/Desktop/anly-501-project-HuitingSong-1/501-project-website/data/cleaned data/countword.csv")

 - Step 10: Opinion Mining
 
 Opinion mining is an approach to natural language processing (NLP) that identifies the emotional tone behind a body of text. This is a popular way for organizations to determine and categorize opinions about a product, service, or idea. In addition to identifying sentiment, opinion mining can extract the polarity (or the amount of positivity and negativity), subject and opinion holder within the text. Furthermore, sentiment analysis can be applied to varying scopes such as document, paragraph, sentence and sub-sentence levels.

 Since I would like to analysis the attitude people toward the energy consumption, opinion mining is an useful methods to conduct the result. 


In [ ]:
# define the function
def getSentiments(df):
   sid = SentimentIntensityAnalyzer()
   tweet_str = ""
   tweetscore = []
   for tweet in df['text']:
       tweet_str = tweet_str + " " + tweet
       score = sid.polarity_scores(tweet_str)
       tweetscore.append(score)
   return tweetscore

#call the function above to see the result score and form a data frame to record
sentiment= getSentiments(twitter)
texts= pd.DataFrame(twitter.text)
stmscore= pd.DataFrame.from_dict(sentiment)
stmscore.head()

#relate the text and score for better view
txtscore= pd.concat([texts,stmscore],axis=1)
txtscore.head()

#export the csv for future analysis
txtscore.to_csv("/Users/apple/Desktop/anly-501-project-HuitingSong-1/501-project-website/data/cleaneddata/pystmscore.csv")

#### Dataset "twitterR.csv" which is grabbed through python

For Twitter API in R, I used twitter to scratch keywords like **"oil","gas","solar power","wind power"** to scratch users’ attitudes about these four kinds of energy. I want to search different types of energy in order to analysis people attitude toward different enegy, and what they think about two different kind of energy (renewable energy and convential energy).

 - Step 1: load the dataset

In [ ]:
twtr = pd.read_csv("/Users/apple/Desktop/anly-501-project-HuitingSong-1/501-project-website/data/raw data/twitterR.csv")
twtr.head()

 - Step 2: Find the dataset with NA missing value
 
 Since R collects lots of basic infor of users and tweet situation, we just need to check whether there are missing value in text variable. Through checking, there is no missing value in the text.


In [ ]:
twtr["text"].isnull().sum()

 - Step 3: Pre-processing the text info in the dataset
 
 Since the raw data contains lots of puntuations, urls, commas, numbers, highercase, and some other things which will influence our analysis and tokenization. Therefore, we need to remove the unnecessary things. Since this semester, I take Natural Language Processing course, and learn how to use **pipline** to process the text information. So, in this part, I will the method I learn from NLP. 
 
 Through pipline, the things which will influence our analysis will be cleaned and replaced. And then, we will need to transform the sentence from text data to tokens which is a list of words through tokenizer.


In [ ]:
import re
import spacy
from spacy.language import Language

pipeline = spacy.load('en_core_web_sm')

# http://emailregex.com/
email_re = r"""(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*|"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])"""

# replace = [ (pattern-to-replace, replacement),  ...]
replace = [
    (r"<a[^>]*>(.*?)</a>", r"\1"),  # Matches most URLs
    (email_re, "email"),            # Matches emails
    (r"(?<=\d),(?=\d)", ""),        # Remove commas in numbers
    (r"\d+", "number"),              # Map digits to special token <numbr>
    (r"[\t\n\r\*\.\@\,\-\/]", " "), # Punctuation and other junk
    (r"\s+", " ")                   # Stips extra whitespace
]

twitter_sentences = []
for i, d in enumerate(twtr['text']):
    for repl in replace:
        d = re.sub(repl[0], repl[1], d)
    twitter_sentences.append(d)

@Language.component("lab04Preprocessor")
def ng20_preprocess(doc):
    tokens = [token for token in doc 
              if not any((token.is_stop, token.is_punct))]
    tokens = [token.lemma_.lower().strip() for token in tokens]
    tokens = [token for token in tokens if token]
    return " ".join(tokens)


pipeline.add_pipe("lab04Preprocessor")

 - Step 6: Pass data through spacy pipeline

In [ ]:
docs = []
for sent in twitter_sentences:
    docs.append(pipeline(sent))

result = pd.DataFrame(docs)
result.head()

 - Step 7: Backup the processed data

In [ ]:
result.to_csv("/Users/apple/Desktop/anly-501-project-HuitingSong-1/501-project-website/data/cleaned data/ppresult.csv")